In [300]:
from math import exp, log
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt

In [301]:
max_speed = 20
max_accel = 12
min_accel = -max_accel
dt = 3 / 1000
tgt_dist = 60
noise = 0.2
drag = 0.8

In [302]:
def sign(x):
    if x < 0:
        return -1
    elif x > 0:
        return 1
    else:
        return 0

In [303]:
def lognorm(sigma):
    z = np.random.normal(0, sigma)
    return np.exp(z)

In [304]:
def sim(power, max_t=10):
    t = np.arange(0, max_t, dt)
    x, v, a, p = np.zeros_like(t), np.zeros_like(t), np.zeros_like(t), np.zeros_like(t)

    for i in range(t.size):
        if i == 0:
            x[i] = 0
            v[i] = 0
            a[i] = 0
        else:
            x[i] = x[i - 1] + v[i - 1] * dt
            v[i] = v[i - 1] + a[i - 1] * dt

            p[i] = power(t[i], x[i - 1], v[i - 1], a[i - 1])
            p[i] = min(max(p[i], -1), 1)

            back_emf = -max_accel * v[i - 1] / max_speed
            ad = p[i] * max_accel + back_emf
            d = -drag * sign(v[i])

            stopped = sign(v[i - 1]) * sign(v[i]) != 1
            if stopped and abs(ad) < abs(drag):
                a[i] = 0
                v[i] = 0
            else:
                err = lognorm(noise * np.sqrt(dt))
                a[i] = (ad + d) * err

    return t, x, v, a, p

In [305]:
def plot_curves(all_arrays):
    t, x, v, a, p = all_arrays
    plt.figure(figsize=(20, 12))
    plt.subplot(411)
    plt.hlines(tgt_dist, t.min(), t.max(), colors='orange')
    plt.plot(t, x, label='x')
    plt.legend()
    plt.subplot(412)
    plt.plot(t, v, label='v')
    plt.legend()
    plt.subplot(413)
    plt.plot(t, a, label='a')
    plt.legend()
    plt.subplot(414)
    plt.plot(t, p, label='p')
    plt.legend()
    plt.show()

In [306]:
# determine ks and kv
trial_pow = np.linspace(0, 0.25, 10)
term_vel = np.ones_like(trial_pow)
for i in range(trial_pow.size):
    ac = sim(lambda *args: trial_pow[i])
    t, x, v, a, p = ac
    term_vel[i] = v[-1]

m = term_vel > 0.5
kv, ks = np.polyfit(term_vel[m], trial_pow[m], 1)

In [ ]:
v = np.array([1,2,3])
p = kv*v + ks
plt.plot(trial_pow, term_vel, '.', label='sim')
plt.plot(p, v, '.', label='fit')
plt.legend()
plt.show()

In [ ]:
extra_pow = np.linspace(0, 0.5, 20)
mean_accel = np.ones_like(extra_pow)

for i in range(extra_pow.size):
    ac = sim(lambda t, x, v, a: ks + kv * v + extra_pow[i])
    t, x, v, a, p = ac
    m = (0 < p) & (p < 1)
    mean_accel[i] = a[m].mean()

plt.plot(extra_pow, mean_accel, '.')
fit = np.polyfit(mean_accel, extra_pow, 1)
assert abs(fit[1]) < 1e-3
ka = fit[0]

In [309]:
eks = ks * lognorm(0.03)
ekv = kv * lognorm(0.03)
eka = ka * lognorm(0.03)

In [ ]:
ac = sim(lambda t, x, v, a: -ks + kv * v + ka * -2)
plot_curves(ac)

In [311]:
back_emf = 1 / (ka * max_speed)
drag = ks / ka

With $a=-k_v$ and $b=-k_s$:
$$v' = a v +b$$
$$v = c \exp(a t) - b / a$$
$$v'=ac \exp(a t)$$
$$ av + b = a(c \exp(a t) - b / a) + b$$

Solve for $c$:
$$v(0)=c\exp(0)-b/a$$
$$c=v(0)+b/a$$

Solve for $x$ stopping at $t$
$$0=c \exp(a t) - b/a$$
$$t=\frac1{a}\log\left(\frac{b}{ac}\right)$$
$$\int_0^t v(s)\ ds = \int_0^t c \exp(a t) - b / a\ dt = \frac{c \exp(a t) - bt}a$$


In [312]:
def est_stopping_point(t0, x0, v0, stopping_power_pct):
    s = sign(v0)
    if s == 0:
        return t0, x0

    a, b = -1 / (ka * max_speed), -1 * (ks + stopping_power_pct) / ka
    c = s * v0 + b/a
    t = log(b / (a * c)) / a # when coasting will stop
    d = (c * exp(a * t) - c - b * t) / a # where coasting will stop
    return t0 + t, x0 + s * d

In [ ]:
print(est_stopping_point(0, 0, 8, 1))
print(est_stopping_point(0, 0, -8, 1))

In [ ]:
stopping_power_pct = 1
xest = None


def f(t, x, v, a):
    global xest
    if t < 1.5:
        _, xest = est_stopping_point(t, x, v, stopping_power_pct)
        return 1
    elif abs(v) > 0.1:
        return -sign(v) * stopping_power_pct
    else:
        return 0


ac = sim(f)
tsim, xsim, _, _, _ = ac
print(f'est: {xest:.2f} ~ sim: {xsim[-1]:.2f}')
assert abs(xsim[-1] - xest) < 0.1

In [315]:
def stopping_loc(x, v, a_mag):
    a = -sign(v) * a_mag
    d = -v * v / (2 * a)
    return d

In [316]:
def direction_test(a, b, atol, btol, test):
    if abs(a) < atol:
        a = 0
    
    if abs(b) < btol:
        b = 0

    return sign(a) * sign(b) == test

In [ ]:
kv, ks, ka

In [ ]:
nom_accel = 10
astop0 = 10.5
astop1 = 5
max_jerk = 100

prev_t = 0

def f(t, x, v, a):
    global prev_t
    dt = t - prev_t

    d = tgt_dist - x

    s0 = stopping_loc(x, v, astop0)
    s1 = stopping_loc(x, v, astop1)
    s0, s1 = min(s0, s1), max(s0, s1)
    if s0 < d < s1:
        au = -v * v / (2 * d)
    else:
        au = sign(d) * nom_accel

    min_a, max_a = a - max_jerk * dt, a + max_jerk * dt
    next_a = min(max(au, min_a), max_a)

    prev_t = t

    s = sign(v)
    return ka * next_a + kv * v# + ks * s

ac = sim(f)
plot_curves(ac)